In [1]:
!unzip "/content/drive/MyDrive/horse_human_cnn.zip"

Archive:  /content/drive/MyDrive/horse_human_cnn.zip
   creating: horse_human_cnn/
   creating: horse_human_cnn/test/
   creating: horse_human_cnn/test/horses/
  inflating: horse_human_cnn/test/horses/horse1-000.png  
  inflating: horse_human_cnn/test/horses/horse1-105.png  
  inflating: horse_human_cnn/test/horses/horse1-122.png  
  inflating: horse_human_cnn/test/horses/horse1-127.png  
  inflating: horse_human_cnn/test/horses/horse1-170.png  
  inflating: horse_human_cnn/test/horses/horse1-204.png  
  inflating: horse_human_cnn/test/horses/horse1-224.png  
  inflating: horse_human_cnn/test/horses/horse1-241.png  
  inflating: horse_human_cnn/test/horses/horse1-264.png  
  inflating: horse_human_cnn/test/horses/horse1-276.png  
  inflating: horse_human_cnn/test/horses/horse1-298.png  
  inflating: horse_human_cnn/test/horses/horse1-335.png  
  inflating: horse_human_cnn/test/horses/horse1-384.png  
  inflating: horse_human_cnn/test/horses/horse1-411.png  
  inflating: horse_human_cnn

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,AveragePooling2D,Rescaling

from tensorflow.keras.utils import image_dataset_from_directory

##Reading Images

In [3]:
train_path ="/content/horse_human_cnn/train"
test_path ="/content/horse_human_cnn/test"

train_data = image_dataset_from_directory(
    directory = train_path,
    image_size=(180,180),
    batch_size = 32
)

test_data = image_dataset_from_directory(
    directory = test_path,
    image_size =(180,180),
    batch_size = 32
)

Found 1037 files belonging to 2 classes.
Found 256 files belonging to 2 classes.


In [4]:
train_data.class_names

['horses', 'humans']

## Training

In [5]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=(180,180,3)))
model.add(Conv2D(16, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])

model.fit(train_data, validation_data=test_data,epochs=5)

Epoch 1/5
33/33 [==============================] - 17s 162ms/step - loss: 1.9547 - accuracy: 0.8129 - val_loss: 2.6739 - val_accuracy: 0.5625
Epoch 2/5
33/33 [==============================] - 6s 170ms/step - loss: 0.0470 - accuracy: 0.9836 - val_loss: 3.3683 - val_accuracy: 0.5781
Epoch 3/5
33/33 [==============================] - 5s 117ms/step - loss: 0.0176 - accuracy: 0.9971 - val_loss: 3.0949 - val_accuracy: 0.5938
Epoch 4/5
33/33 [==============================] - 5s 115ms/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 3.0876 - val_accuracy: 0.6133
Epoch 5/5
33/33 [==============================] - 6s 130ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 3.7991 - val_accuracy: 0.5859


## Data Augmentation

In [6]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

In [7]:
model2 = Sequential()
model2.add(RandomFlip("horizontal", input_shape=(180,180,3)))
model2.add(RandomRotation(0.1))
model2.add(RandomZoom(0.1))
model2.add(Rescaling(1./255))
model2.add(Conv2D(16,(3,3), padding ='same'))
model2.add(MaxPooling2D())
model2.add(Conv2D(64, (3,3), padding='same'))
model2.add(MaxPooling2D())
model2.add(Flatten())
model2.add(Dense(128,activation="relu"))
model2.add(Dense(1,activation="sigmoid"))

model2.compile(loss="binary_crossentropy", optimizer ="adam", metrics=["accuracy"])

model2.fit(train_data, validation_data=test_data, epochs=5)

Epoch 1/5
33/33 [==============================] - 7s 116ms/step - loss: 3.2568 - accuracy: 0.6663 - val_loss: 6.3217 - val_accuracy: 0.5234
Epoch 2/5
33/33 [==============================] - 7s 154ms/step - loss: 0.8183 - accuracy: 0.7869 - val_loss: 1.3646 - val_accuracy: 0.7539
Epoch 3/5
33/33 [==============================] - 5s 115ms/step - loss: 0.1946 - accuracy: 0.9238 - val_loss: 2.5967 - val_accuracy: 0.6133
Epoch 4/5
33/33 [==============================] - 6s 161ms/step - loss: 0.1590 - accuracy: 0.9325 - val_loss: 2.3213 - val_accuracy: 0.6367
Epoch 5/5
33/33 [==============================] - 5s 117ms/step - loss: 0.1602 - accuracy: 0.9306 - val_loss: 3.1429 - val_accuracy: 0.5977


## Transfer Learning

In [8]:
from tensorflow.keras.applications import VGG16

In [9]:
vgg16 = VGG16(weights='imagenet', include_top=False)
vgg16.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

In [10]:
model3 = Sequential()
model3.add(Rescaling(1./255, input_shape = (180,180,3)))
model3.add(vgg16)
model3.add(Flatten())
model3.add(Dense(128, activation = "relu"))
model3.add(Dense(1, activation = "sigmoid"))

vgg16.trainable = False

model3.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

model3.fit(train_data, validation_data=test_data,epochs=5)

Epoch 1/5
33/33 [==============================] - 14s 210ms/step - loss: 0.3251 - accuracy: 0.8872 - val_loss: 0.0897 - val_accuracy: 0.9609
Epoch 2/5
33/33 [==============================] - 7s 194ms/step - loss: 0.0118 - accuracy: 0.9981 - val_loss: 0.0811 - val_accuracy: 0.9688
Epoch 3/5
33/33 [==============================] - 6s 152ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.1210 - val_accuracy: 0.9453
Epoch 4/5
33/33 [==============================] - 9s 240ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.1290 - val_accuracy: 0.9453
Epoch 5/5
33/33 [==============================] - 6s 153ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.1235 - val_accuracy: 0.9531


## Predict Image

In [11]:
from tensorflow.keras.preprocessing import image

In [12]:
img = image.load_img("/content/horse_human_cnn/test/horses/horse1-264.png", target_size =(180,180))
img_array = image.img_to_array(img)

In [13]:
img_batch = np.expand_dims(img_array,axis=0)

In [14]:
img_batch.shape

(1, 180, 180, 3)

In [15]:
result = model3.predict(img_batch)
result

1/1 [==============================] - 1s 644ms/step


array([[2.948733e-05]], dtype=float32)

In [16]:
if result >= 0.5:
  print("human")
else:
  print("horse")

horse


In [17]:
# function to predict the image

def predict_image(img_path):
  img = image.load_img(img_path, target_size = (180,180))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  result = model3.predict(img_batch)
  if result >= 0.5:
    return "human"
  else:
    return "horse"

In [18]:
 predict_image("/content/horse_human_cnn/test/horses/horse6-544.png")

1/1 [==============================] - 0s 20ms/step


'horse'

In [19]:
predict_image("/content/horse_human_cnn/test/humans/valhuman05-27.png")

1/1 [==============================] - 0s 19ms/step


'human'

# Save model

In [21]:
model3.save("horse_human_classifier.h5")

In [22]:
from google.colab import files
files.download("/content/horse_human_classifier.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>